# Tutorial 06, case 1b: Poisson problem with distributed control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} (y - y_d)^2 dx + \frac{\alpha}{2} \int_{\Omega} u^2 dx$$
s.t.
$$\begin{cases}
- \Delta y = f + u     & \text{in } \Omega\\
         y = 1         & \text{on } \partial\Omega
\end{cases}$$

where
$$\begin{align*}
& \Omega               & \text{unit square}\\
& u \in L^2(\Omega)    & \text{control variable}\\
& y \in H^1(\Omega)    & \text{state variable}\\
& \alpha > 0           & \text{penalization parameter}\\
& y_d                  & \text{a piecewise constant desired state}\\
& f                    & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach.

The test case is from section 5.1 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace
from dolfinx.fem import (apply_lifting, assemble_matrix, assemble_matrix_block, assemble_scalar,
                         assemble_vector, assemble_vector_block, BlockVecSubVectorWrapper,
                         create_vector_block, locate_dofs_topological, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/rectangle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.create_connectivity_all()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1 = boundaries.indices[boundaries.values == 1]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Function spaces

In [ ]:
Y = FunctionSpace(mesh, ("Lagrange", 1))
U = FunctionSpace(mesh, ("Lagrange", 1))
Q = Y.clone()

### Trial and test functions

In [ ]:
(y, u, p) = (TrialFunction(Y), TrialFunction(U), TrialFunction(Q))
(z, v, q) = (TestFunction(Y), TestFunction(U), TestFunction(Q))

 ### Problem data

In [ ]:
alpha = 0.01
y_d_1 = 1.0
y_d_2 = 0.6
ff = Constant(mesh, 0.)
bc0 = Function(Y)
bc0.interpolate(lambda x: np.zeros(x.shape[1]))
bc1 = Function(Y)
bc1.interpolate(lambda x: np.ones(x.shape[1]))

### Optimality conditions

In [ ]:
a = [[y * z * dx, None, inner(grad(p), grad(z)) * dx],
     [None, alpha * u * v * dx, - p * v * dx],
     [inner(grad(y), grad(q)) * dx, - u * q * dx, None]]
f = [y_d_1 * z * dx(1) + y_d_2 * z * dx(2),
     None,
     ff * q * dx]
a[2][2] = Constant(mesh, 0.) * p * q * dx
f[1] = Constant(mesh, 0.) * v * dx
bdofs_Y_1 = locate_dofs_topological((Y, Y), mesh.topology.dim - 1, boundaries_1)
bdofs_Q_1 = locate_dofs_topological((Q, Y), mesh.topology.dim - 1, boundaries_1)
bc = [DirichletBC(bc1, bdofs_Y_1, Y),
      DirichletBC(bc0, bdofs_Q_1, Q)]

### Solution

In [ ]:
(y, u, p) = (Function(Y), Function(U), Function(Q))

### Cost functional

In [ ]:
J = (0.5 * inner(y - y_d_1, y - y_d_1) * dx(1) + 0.5 * inner(y - y_d_2, y - y_d_2) * dx(2)
     + 0.5 * alpha * inner(u, u) * dx)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = replace(a[2][0], {q: z})
f_state = replace(f[2], {q: z})
bc_state = [bc[0]]

In [ ]:
# Assemble the linear system for the state
A_state = assemble_matrix(a_state, bcs=bc_state)
A_state.assemble()
F_state = assemble_vector(f_state)
apply_lifting(F_state, [a_state], [bc_state])
F_state.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
set_bc(F_state, bc_state)

In [ ]:
# Solve
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, y.vector)
y.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
J_uncontrolled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.24)

In [ ]:
plot(y, title="uncontrolled state")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = assemble_matrix_block(a, bcs=bc)
A.assemble()
F = assemble_vector_block(f, a, bcs=bc)

In [ ]:
# Solve
yup = create_vector_block(f)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, yup)
yup.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(yup, [Y.dofmap, U.dofmap, Q.dofmap]) as yup_wrapper:
    for yup_wrapper_local, component in zip(yup_wrapper, (y, u, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = yup_wrapper_local

In [ ]:
J_controlled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 0.158485065)

In [ ]:
plot(y, title="state")

In [ ]:
plot(u, title="control")

In [ ]:
plot(p, title="adjoint")